In [1]:
import scipy
import numpy as np
from typing import Optional, Union
from loguru import logger
import sys
import time

import matplotlib.pyplot as plt
from scipy.stats import levy_stable

from functools import partial

import torch
import torchquad

import pyro.distributions as dist
from pyro.distributions import Stable
import pyro

from torchquad import set_up_backend, Trapezoid, Simpson, Boole, MonteCarlo, VEGAS, GaussLegendre, IntegrationGrid
from torchquad.utils.set_precision import set_precision

from emsim.stable_dist import Batch1DIntegrator
from emsim.stable_dist.pdf import _V, _theta_0, _zeta, stable_standard_density
from emsim.stable_dist.pdf_2 import standard_density_2
from emsim.stable_dist.pdf_old import stable_standard_density_old
# from emsim.stable_dist.pyro_dist import StableWithLogProb

In [15]:
class StableWithLogProb(Stable):
    def __init__(self, stability, skew, scale=1, loc=0, coords="S0", validate_args=None):
        super().__init__(stability, skew, scale, loc, coords, validate_args)
        self._integrator = Batch1DIntegrator()
        self._integrate_fn = None
        
    def log_prob(self, value, integrator_N_gridpoints=501):
        if self._validate_args:
            self._validate_sample(value)

        value = (value - self.loc) / self.scale
        if self.coords == "S0":
            zeta = _zeta(self.stability, self.skew)
            value = value - zeta
        p = stable_standard_density_old(
            value,
            self.stability,
            self.skew,
            integrator_N_gridpoints=integrator_N_gridpoints,
            integrator=self._integrator
        )
        p = p / self.scale
        return p.log()
    
    def log_prob_2(self, value):
        value = (value - self.loc) / self.scale
        p = standard_density_2(value, self.stability, self.skew)
        p = p / self.scale
        return p.log()
    
    def log_prob_integrate_once(self, value, integrator_N_gridpoints=501):
        if self._integrate_fn is None:
            self._integrate_fn = self._integrator.get_jit_compiled_integrate(
                dim=1, N=integrator_N_gridpoints
            )
        
        value = (value - self.loc) / self.scale
        if self.coords == "S0":
            zeta = _zeta(self.stability, self.skew)
            value = value - zeta
        p = stable_standard_density(
            value,
            self.stability,
            self.skew,
            integrate_fn=self._integrate_fn
        )
        p = p / self.scale
        return p.log()

stable = StableWithLogProb(1.0, 0.0, scale=3., loc=0.)

In [25]:
# stable = StableWithLogProb(1.0, 0.0, scale=3., loc=0., coords="S0")
stable.stability = torch.tensor(1.9)
stable.skew = torch.tensor(0.7)
x = stable.sample((10,))
# x = torch.tensor(0.0, dtype=torch.float64)
# print(f"{stable.coords=}")
# print(f"{x=}")

t0 = time.time()
print(f"pyro likelihood = {stable.log_prob(x).exp()}")
print(f"Time = {time.time() - t0}")

t0 = time.time()
print(f"pyro likelihood (N={(N:=1001)}) = {stable.log_prob(x, integrator_N_gridpoints=N).exp()}")
print(f"Time = {time.time() - t0}")

t0 = time.time()
print(f"pyro likelihood integrate once = {stable.log_prob_integrate_once(x).exp()}")
print(f"Time = {time.time() - t0}")


levy_stable.parameterization = "S1"
t0 = time.time()
print(f"scipy S1 likelihood = {levy_stable.pdf(x.numpy(), stable.stability.item(), stable.skew.item(), loc=stable.loc.item(), scale=stable.scale.item())}")
print(f"Time = {time.time() - t0}")

levy_stable.parameterization = "S0"
t0 = time.time()
print(f"scipy S0 likelihood = {levy_stable.pdf(x.numpy(), stable.stability.item(), stable.skew.item(), loc=stable.loc.item(), scale=stable.scale.item())}")
print(f"Time = {time.time() - t0}")

pyro likelihood = tensor([0.0910, 0.0654, 0.0535, 0.0820, 0.0891, 0.0109, 0.0791, 0.0851, 0.0866,
        0.0714], dtype=torch.float64)
Time = 0.011824846267700195
pyro likelihood (N=1001) = tensor([0.0910, 0.0654, 0.0535, 0.0820, 0.0891, 0.0109, 0.0791, 0.0851, 0.0866,
        0.0714], dtype=torch.float64)
Time = 0.004446268081665039
pyro likelihood integrate once = tensor([0.0910, 0.0654, 0.0535, 0.0820, 0.0891, 0.0109, 0.0791, 0.0851, 0.0866,
        0.0714], dtype=torch.float64)
Time = 0.0018570423126220703
scipy S1 likelihood = [0.09265054 0.06989632 0.04916155 0.07842857 0.09121308 0.01281841
 0.08270244 0.08193286 0.08361902 0.067191  ]
Time = 0.011184930801391602
scipy S0 likelihood = [0.09104304 0.06540118 0.05346387 0.08199202 0.08909369 0.01086492
 0.0790832  0.08514577 0.08662685 0.07140776]
Time = 0.009859800338745117


In [27]:
x = stable.sample((1000,))

In [21]:
%%timeit
stable.log_prob(x)

100 ms ± 4.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%%timeit
stable.log_prob_integrate_once(x)

117 ms ± 7.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%%timeit
levy_stable.pdf(x.numpy(), stable.stability.item(), stable.skew.item(), loc=stable.loc.item(), scale=stable.scale.item())

710 ms ± 6.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
temp = x
zeta = _zeta(torch.tensor(1.2), torch.tensor(.8))
temp = (temp - 2) / 3
temp = temp - zeta
stable_standard_density_old(temp, torch.tensor(1.2), torch.tensor(0.8))

/Users/mawright/micromamba/envs/emsim/lib/python3.11/site-packages/autoray/autoray.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return get_lib_fn(backend, fn)(*args, **kwargs)
/Users/mawright/micromamba/envs/emsim/lib/python3.11/site-packages/torchquad/integration/utils.py:255: UserWarning: DEPRECATION WARNING: In future versions of torchquad, an array-like object will be returned.
  warnings.warn(


tensor([0.2581], dtype=torch.float64)

In [ ]:
zeta = -2.4621
x0 = .24086666
xi = 0.98749974134
c2 = 0.8598012431

In [47]:
dist.Cauchy(2., 3.).log_prob(torch.tensor(2.7226)).exp()

tensor(0.1003)

In [53]:
def levy_stable_pdf_with_params(x, pyro_dist: Stable):
    return levy_stable.pdf(x, pyro_dist.stability.item(), pyro_dist.skew.item(), loc=pyro_dist.loc.item(), scale=pyro_dist.scale.item())

In [56]:
scipy.integrate.quad(partial(levy_stable_pdf_with_params, pyro_dist=stable), -np.inf, np.inf)

(1.0000000170194838, 1.464015531027485e-08)

In [52]:
integrator = GaussLegendre()
integrator.integrate(pyro_dist)

tensor(10558.7266, dtype=torch.float64)

In [11]:
stable.log_prob(value).exp().item() / stable.scale.item()

0.10690703988075256

In [12]:
levy_stable.parameterization = "S0"
levy_stable.pdf(value.item(), stable.stability.item(), stable.skew.item(), loc=stable.loc.item(), scale=stable.scale.item())

0.12045776512240929

In [11]:
stable.scale

tensor(1.)

In [10]:
p.exp()

NameError: name 'p' is not defined

In [2]:
set_up_backend("torch", "float64")

11:07:06|TQ-WARNING| Error enabling CUDA. cuda.is_available() returned False. CPU will be used.


In [3]:
integration_domain_eps = 0

In [4]:
## Simple integrand for example

In [5]:
f = torch.rand((3,))
x0 = torch.randn((3,))
xT = x0 + torch.rand((3,)) * 2
gl = GaussLegendre()

In [6]:
def true_result(x0, xT, f):
    def antiderivative(x, f):
        return (1 / f) * torch.sin(f * x)
    return antiderivative(xT, f) - antiderivative(x0, f)
true_result(x0, xT, f)

tensor([1.4118, 0.2026, 0.4939])

In [7]:
def test_simple_integrand_1d(x, i):
    return torch.cos(f[i] * x)

In [8]:
onebyone_result = []
for i in range(3):
    out = gl.integrate(partial(test_simple_integrand_1d, i=i), 1, 101, torch.tensor([[x0[i], xT[i]]]))
    onebyone_result.append(out)
print(f"{onebyone_result=}")

onebyone_result=[tensor(1.4118), tensor(0.2026), tensor(0.4939)]


/Users/mawright/micromamba/envs/emsim/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1682343668887/work/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/mawright/micromamba/envs/emsim/lib/python3.11/site-packages/autoray/autoray.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return get_lib_fn(backend, fn)(*args, **kwargs)
/Users/mawright/micromamba/envs/emsim/lib/python3.11/site-packages/torchquad/integration/utils.py:255: UserWarning: DEPRECATION WARNING: In future versions of torchquad, an array-like object will be returned.
  warnings.warn(


In [9]:
def test_simple_integrand(x):
    return torch.cos(f * x)

In [10]:
b1d = Batch1DIntegrator()
b1d.integrate(test_simple_integrand, 1, 101, torch.stack([x0, xT], -1))

tensor([1.4118, 0.2026, 0.4939])

In [11]:
dim = 3
# alpha = torch.rand((dim,)) * 2
# beta = torch.rand((dim,)) * 2 - 1

# alpha = alpha.double()
# beta = beta.double()

alpha = torch.tensor([.3497, .1555, .5036])
beta = torch.tensor([.6367, .6049, .8645])

theta_0 = _theta_0(alpha, beta)
domain = torch.stack([-theta_0, theta_0.new_full(theta_0.shape, torch.pi / 2)], -1)

# x = dist.Stable(alpha, beta).sample().abs()
value = torch.tensor([11.8309, 0.5035, 2.5369])
print(f"{value=}")

x=tensor([11.8309,  0.5035,  2.5369])


In [12]:
stable_standard_density_old(value, alpha, beta)

tensor([0.0064, 0.0923, 0.0735])

In [15]:
scipy_pdf_values = []
for x_i, a_i, b_i in zip(value.numpy(), alpha.numpy(), beta.numpy()):
    out = levy_stable.pdf(x_i, a_i, b_i)
    scipy_pdf_values.append(out)
print(scipy_pdf_values)

[0.0064064984508115895, 0.0922659518550362, 0.07351105417998094]


In [214]:
def test_integrand_1d(theta, i):
    V = _V(theta, alpha[i], beta[i], theta_0[i])
    # print(f"{V=}")
    out = V * torch.exp(- value[i] ** (alpha[i] / (alpha[i] - 1)) * V)
    # print(f"{out=}")
    return out

In [216]:
gl = GaussLegendre()
onebyone_result = []
for i in range(dim):
    out = gl.integrate(
        partial(test_integrand_1d, i=i), 1, 101, torch.tensor([[-theta_0[i], torch.pi / 2]]))
    onebyone_result.append(out)
    print(onebyone_result)

2023-10-03 10:12:54.360 | DEBUG    | torchquad.integration.utils:_setup_integration_domain:112 - Setting up integration domain.
2023-10-03 10:12:54.360 | DEBUG    | torchquad.integration.base_integrator:_check_inputs:105 - Checking inputs to Integrator.
2023-10-03 10:12:54.361 | DEBUG    | torchquad.integration.grid_integrator:calculate_grid:122 - Creating a grid for GaussLegendre to integrate a function with 101 points over tensor([[-1.0628,  1.5708]]).
2023-10-03 10:12:54.362 | DEBUG    | torchquad.integration.integration_grid:_check_inputs:110 - Checking inputs to IntegrationGrid.
2023-10-03 10:12:54.363 | DEBUG    | torchquad.integration.integration_grid:__init__:66 - Creating 1-dimensional integration grid with 101 points over tensor([[-1.0628,  1.5708]])
2023-10-03 10:12:54.366 | DEBUG    | torchquad.integration.integration_grid:__init__:95 - Grid mesh width is tensor([0.0016])
2023-10-03 10:12:54.368 | INFO     | torchquad.integration.integration_grid:__init__:103 - Integration 

/Users/mawright/micromamba/envs/emsim/lib/python3.11/site-packages/autoray/autoray.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return get_lib_fn(backend, fn)(*args, **kwargs)
/Users/mawright/micromamba/envs/emsim/lib/python3.11/site-packages/torchquad/integration/utils.py:255: UserWarning: DEPRECATION WARNING: In future versions of torchquad, an array-like object will be returned.
  warnings.warn(


In [211]:
def test_integrand(theta):
    V = _V(theta, alpha, beta, theta_0)
    # print(f"{V=}")
    out =  V * torch.exp(- value ** (alpha / (alpha - 1)) * V)
    # print(f"{out=}")
    return out

In [212]:
gl = GaussLegendre()
gl.integrate(test_integrand, dim, 101, domain)

2023-10-03 10:11:36.012 | DEBUG    | torchquad.integration.utils:_setup_integration_domain:112 - Setting up integration domain.
2023-10-03 10:11:36.014 | DEBUG    | torchquad.integration.base_integrator:_check_inputs:105 - Checking inputs to Integrator.
2023-10-03 10:11:36.015 | DEBUG    | torchquad.integration.grid_integrator:calculate_grid:122 - Creating a grid for GaussLegendre to integrate a function with 101 points over tensor([[-1.0628,  1.5708],
        [-0.9623,  1.5708],
        [-1.4266,  1.5708]]).
2023-10-03 10:11:36.017 | DEBUG    | torchquad.integration.integration_grid:_check_inputs:110 - Checking inputs to IntegrationGrid.
2023-10-03 10:11:36.018 | DEBUG    | torchquad.integration.integration_grid:__init__:66 - Creating 3-dimensional integration grid with 101 points over tensor([[-1.0628,  1.5708],
        [-0.9623,  1.5708],
        [-1.4266,  1.5708]])
2023-10-03 10:11:36.019 | DEBUG    | torchquad.integration.integration_grid:__init__:95 - Grid mesh width is tensor([

tensor([11.7538,  5.4102, 10.2102])

In [16]:
def batch_linspace_with_grads(start: torch.Tensor, stop: torch.Tensor, N: Union[torch.Tensor, int]) -> torch.Tensor:
    linspace = torch.linspace(0.0, 1.0, N).unsqueeze(0)
    start = start.reshape(-1, 1)
    stop = stop.reshape(-1, 1)
    grid = linspace * (stop - start) + start
    return grid

In [217]:
b1d = Batch1DIntegrator()
b1d.integrate(test_integrand, 1, 101, domain)

2023-10-03 10:13:05.814 | DEBUG    | __main__:_resize_roots:18 - resize_roots output: tensor([[-1.0624, -0.9619, -1.4262],
        [-1.0608, -0.9604, -1.4244],
        [-1.0580, -0.9577, -1.4212],
        [-1.0539, -0.9537, -1.4165],
        [-1.0485, -0.9486, -1.4104],
        [-1.0419, -0.9423, -1.4029],
        [-1.0341, -0.9347, -1.3940],
        [-1.0250, -0.9260, -1.3837],
        [-1.0147, -0.9161, -1.3720],
        [-1.0032, -0.9050, -1.3589],
        [-0.9905, -0.8928, -1.3444],
        [-0.9766, -0.8794, -1.3286],
        [-0.9615, -0.8649, -1.3114],
        [-0.9453, -0.8493, -1.2929],
        [-0.9279, -0.8326, -1.2731],
        [-0.9094, -0.8147, -1.2520],
        [-0.8897, -0.7958, -1.2297],
        [-0.8690, -0.7759, -1.2061],
        [-0.8472, -0.7549, -1.1812],
        [-0.8243, -0.7329, -1.1552],
        [-0.8004, -0.7099, -1.1280],
        [-0.7755, -0.6860, -1.0996],
        [-0.7496, -0.6610, -1.0702],
        [-0.7227, -0.6352, -1.0396],
        [-0.6949, -0.6085,

tensor([1.6720, 0.6985, 1.4850])

In [45]:
alpha = torch.rand((100,)) * 2
alpha = alpha.reshape(100, 1)
beta = torch.rand((100,)) * 2 - 1
beta = beta.reshape(1, 100)
theta_0 = _theta_0(alpha, beta)

In [130]:
alpha = torch.tensor(1.1)
beta = torch.tensor(0.8)
theta_0 = _theta_0(alpha, beta)

In [152]:
value = dist.Stable(alpha, beta, 1, 0).sample().abs()

In [114]:
theta_0

tensor(-1.2503)

In [104]:
value.shape

torch.Size([])

In [196]:
theta_0.shape

torch.Size([100, 1])

In [46]:
V = _V(theta_domain, alpha, beta, theta_0)

NameError: name 'theta_domain' is not defined

In [47]:
test_integrand(theta_domain)

NameError: name 'theta_domain' is not defined

In [48]:
-(value ** (alpha / (alpha - 1)))

tensor([[ -9.3392e+12,  -8.6715e+01,  -3.8614e+00],
        [ -5.1968e-02,  -6.4283e-01,  -8.7479e-01],
        [ -2.1880e+07,  -1.2497e+01,  -2.1481e+00],
        [ -4.5209e+21,  -1.7213e+03,  -9.5418e+00],
        [ -5.2948e-02,  -6.4463e-01,  -8.7553e-01],
        [ -7.4660e+08,  -2.1177e+01,  -2.5200e+00],
        [ -1.3209e+13,  -9.1326e+01,  -3.9224e+00],
        [ -4.5582e-03,  -4.4685e-01,  -7.8359e-01],
        [ -1.4109e-05,  -1.8846e-01,  -6.0336e-01],
        [ -4.5104e+07,  -1.3923e+01,  -2.2195e+00],
        [ -2.3887e+07,  -1.2662e+01,  -2.1566e+00],
        [ -4.5497e-01,  -8.8898e-01,  -9.6500e-01],
        [ -1.6375e-09,  -4.8660e-02,  -4.0045e-01],
        [ -1.3621e-01,  -7.4238e-01,  -9.1376e-01],
        [ -0.0000e+00, -1.5487e-178,  -1.4764e-54],
        [ -2.7766e-04,  -2.9415e-01,  -6.9042e-01],
        [ -1.5470e-04,  -2.6953e-01,  -6.7239e-01],
        [ -3.4777e+10,  -3.7596e+01,  -2.9982e+00],
        [-1.5703e-104,  -3.0833e-16,  -2.0144e-05],
        [ -3

In [49]:
theta_domain

NameError: name 'theta_domain' is not defined

In [50]:
test_integrand(torch.tensor(1.9))

RuntimeError: The size of tensor a (3) must match the size of tensor b (100) at non-singleton dimension 1

In [51]:
gl = GaussLegendre()
simp = Simpson()

In [52]:
gl.integrate(test_integrand, 1, 101, torch.tensor([[-theta_0, np.pi / 2]]))

ValueError: only one element tensors can be converted to Python scalars

In [54]:
alpha

tensor(0.1422)

In [198]:
(torch.pi / 2 - integration_domain_eps).expand(100, 1)

AttributeError: 'float' object has no attribute 'expand'

In [206]:
integration_domain_eps = 1e-5
integration_domain = torch.tensor([[-theta_0 + integration_domain_eps, torch.pi / 2 - integration_domain_eps]])
# integration_domain = torch.cat([-theta_0 + integration_domain_eps, theta_0.new_full(theta_0.shape, torch.pi / 2 - integration_domain_eps)], -1)
simp.integrate(test_integrand, dim=1, N=101, integration_domain=integration_domain)

ValueError: only one element tensors can be converted to Python scalars

In [223]:
_V(-theta_0[0], alpha[0], beta[0], theta_0[0])

tensor([0.])

tensor([0.6378])

In [224]:
_V(-theta_0, alpha, beta, _theta_0(alpha, beta))

tensor([[0.],
        [inf],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [inf],
        [0.],
        [inf],
        [0.],
        [inf],
        [inf],
        [inf],
        [inf],
        [0.],
        [0.],
        [inf],
        [inf],
        [inf],
        [inf],
        [0.],
        [0.],
        [inf],
        [0.],
        [0.],
        [0.],
        [nan],
        [nan],
        [inf],
        [inf],
        [inf],
        [inf],
        [0.],
        [0.],
        [0.],
        [inf],
        [inf],
        [inf],
        [inf],
        [0.],
        [inf],
        [0.],
        [0.],
        [0.],
        [inf],
        [0.],
        [0.],
        [inf],
        [0.],
        [0.],
        [inf],
        [inf],
        [0.],
        [inf],
        [inf],
        [0.],
        [inf],
        [inf],
        [inf],
        [0.],
        [0.],
        [0.],
        [inf],
        [0.],
        [0.],
        [0.],
        [0.],
        [inf],


In [208]:
_V(-theta_0 + integration_domain_eps, alpha, beta, theta_0)

tensor([[1.2803e-03],
        [4.1114e+12],
        [8.8257e-09],
        [1.5827e-11],
        [3.1572e-07],
        [3.1919e-05],
        [2.5811e-03],
        [3.6173e+05],
        [9.9920e-02],
        [7.2503e+06],
        [0.0000e+00],
        [2.5695e+05],
        [1.4039e+27],
        [3.0032e+05],
        [1.7070e+24],
        [1.1427e-02],
        [3.5112e-02],
        [1.5659e+08],
        [1.3804e+07],
        [3.3752e+06],
        [3.3554e+15],
        [3.1054e-03],
        [4.0318e-02],
        [5.3253e+06],
        [1.4053e-01],
        [6.6881e-08],
        [7.1811e-02],
        [       nan],
        [       nan],
        [       inf],
        [3.6134e+09],
        [1.0737e+18],
        [5.6792e+06],
        [1.6812e-17],
        [4.1521e-03],
        [3.5778e-02],
        [6.5117e+07],
        [6.3229e+05],
        [1.8440e+07],
        [2.0306e+12],
        [5.6749e-06],
        [3.4272e+06],
        [2.3123e-05],
        [1.9636e-01],
        [8.4590e-02],
        [1

In [120]:
eps

0.00011920928955078125